This tutorial is automatically generated from the file test/python/tutorials//TestPythonOffLatticeAngiogenesisLiteratePaper.py.



In [ ]:
# Jupyter notebook specific imports 
import matplotlib as mpl 
from IPython import display 
%matplotlib inline


# An Off Lattice Angiogenesis Tutorial
This tutorial demonstrates functionality for modelling 3D off-lattice angiogenesis in a corneal micro
pocket application, similar to that described in [Connor et al. 2015](http://rsif.royalsocietypublishing.org/content/12/110/20150546.abstract).

It is a 3D simulation modelling VEGF diffusion and decay from an implanted pellet using finite element methods and
lattice-free angiogenesis from a large limbal vessel towards the pellet.

# The Test



In [ ]:
import numpy as np
import chaste # Core Chaste functionality
import chaste.cell_based # Chaste Cell Populations
chaste.init() # Initialize MPI and PETSc
import microvessel_chaste # Core Microvessel Chaste functionality
import microvessel_chaste.geometry # Geometry tools
import microvessel_chaste.mesh # Meshing
import microvessel_chaste.population.vessel # Vessel tools
import microvessel_chaste.pde # PDE and solvers
import microvessel_chaste.simulation # Flow and angiogenesis solvers
import microvessel_chaste.visualization # Visualization
from microvessel_chaste.utility import * # Dimensional analysis: bring in all units for convenience
# Set up the test 
chaste.cell_based.SetupNotebookTest()

Set up output file management.



In [ ]:
file_handler = chaste.core.OutputFileHandler("Python/TestOffLatticeAngiogenesisLiteratePaper")
chaste.core.RandomNumberGenerator.Instance().Reseed(12345)

This component uses explicit dimensions for all quantities, but interfaces with solvers which take
non-dimensional inputs. The `BaseUnits` singleton takes time, length and mass reference scales to
allow non-dimensionalisation when sending quantities to external solvers and re-dimensionalisation of
results. For our purposes microns for length and hours for time are suitable base units.



In [ ]:
reference_length = 1.e-6 * metres
reference_time = 3600.0 * seconds
reference_concentration = 1.e-9*mole_per_metre_cubed
BaseUnits.Instance().SetReferenceLengthScale(reference_length)
BaseUnits.Instance().SetReferenceTimeScale(reference_time)
BaseUnits.Instance().SetReferenceConcentrationScale(reference_concentration)

Set up the domain representing the cornea. This is a thin hemispherical shell. We assume some symmetry to
reduce computational expense.



In [ ]:
hemisphere_generator = microvessel_chaste.geometry.MappableGridGenerator3()
radius = 1400.0e-6*metres
thickness = 100.0e-6*metres
num_divisions_x = 10
num_divisions_y = 10
azimuth_angle = 1.0 * np.pi
polar_angle = 0.5 * np.pi
cornea = hemisphere_generator.GenerateHemisphere(radius,
                                                 thickness,
                                                 num_divisions_x,
                                                 num_divisions_y,
                                                 azimuth_angle,
                                                 polar_angle)

We can visualize the part



In [ ]:
scene = microvessel_chaste.visualization.MicrovesselVtkScene3()
scene.SetPart(cornea)
scene.GetPartActorGenerator().SetVolumeOpacity(0.7)
#scene.GetPartActorGenerator().SetVolumeColor((255.0, 255.0, 255.0))
nb_manager = microvessel_chaste.visualization.JupyterNotebookManager()
nb_manager.vtk_show(scene, height=600, width = 1000)

Set up a vessel network, with divisions roughly every 'cell length'. Initially it is straight.
We will map it onto the hemisphere.



In [ ]:
network_generator = microvessel_chaste.population.vessel.VesselNetworkGenerator3()
vessel_length = np.pi * radius
cell_length = 40.0e-6 * metres
origin = microvessel_chaste.geometry.Vertex3(0.0*metres, 4000.0e-6*metres)
scaled_vessel_length = vessel_length.Convert(cell_length)
network  = network_generator.GenerateSingleVessel(vessel_length, origin, int(scaled_vessel_length) + 1, 0)
network.GetNode(0).GetFlowProperties().SetIsInputNode(True);
#network.GetNode(0).GetFlowProperties().SetPressure(Owen11Parameters.mpInletPressure.GetValue("User"))
network.GetNode(network.GetNumberOfNodes()-1).GetFlowProperties().SetIsOutputNode(True)
#network.GetNode(network.GetNumberOfNodes()-1).GetFlowProperties().SetPressure(Owen11Parameters.mpOutletPressure.GetValue("User"))
nodes = network.GetNodes();
for eachNode in nodes:
    loc = eachNode.rGetLocation().rGetLocation()
    scaled_length = reference_length.Convert(vessel_length)
    node_azimuth_angle = float(azimuth_angle * loc.Convert(reference_length)[0]*scaled_length)
    node_polar_angle = float(polar_angle*loc.Convert(reference_length)[1]*scaled_length)
    radius = radius - 0.5*thickness
    new_position = microvessel_chaste.geometry.Vertex3(np.cos(node_azimuth_angle) * np.sin(node_polar_angle)*radius,
                                                                   np.cos(node_polar_angle)*radius,
                                                                   np.sin(node_azimuth_angle) * np.sin(node_polar_angle)*radius)
    eachNode.SetLocation(new_position)

Visualize the network



In [ ]:
scene.SetVesselNetwork(network)
scene.GetVesselNetworkActorGenerator().SetEdgeSize(20.0)
nb_manager.vtk_show(scene, height=600, width = 1000)

In the experimental assay a pellet containing VEGF is implanted near the top of the cornea. We model this
as a fixed concentration of VEGF in a cuboidal region. First set up the vegf sub domain.



In [ ]:
pellet = microvessel_chaste.geometry.Part3()
pellet_side_length = 300.0e-6 * metres
origin = microvessel_chaste.geometry.Vertex3(-150.0e-6 * metres, 900.0e-6 * metres)
pellet.AddCuboid(pellet_side_length, pellet_side_length, 5.0*pellet_side_length, origin)
pellet.Write(file_handler.GetOutputDirectoryFullPath()+"initial_vegf_pellet.vtp",
             microvessel_chaste.geometry.GeometryFormat.VTP, True)

Now make a finite element mesh on the cornea.



In [ ]:
mesh_generator = microvessel_chaste.mesh.DiscreteContinuumMeshGenerator3_3()
mesh_generator.SetDomain(cornea)
mesh_generator.SetMaxElementArea(1e-6 * metres_cubed)
mesh_generator.Update()
mesh = mesh_generator.GetMesh()

We can visualize the mesh



In [ ]:
scene.GetPartActorGenerator().SetVolumeOpacity(0.0)
scene.SetMesh(mesh)
nb_manager.vtk_show(scene, height=600, width = 1000)

Set up the vegf pde. Note the scaling of the refernece concentration to nM to avoid numerical
precision problems.



In [ ]:
vegf_pde = microvessel_chaste.pde.DiscreteContinuumLinearEllipticPde3_3()
#vegf_pde.SetIsotropicDiffusionConstant(Owen11Parameters.mpVegfDiffusivity.GetValue("User"))
#vegf_pde.SetContinuumLinearInUTerm(-1.0*Owen11Parameters.mpVegfDecayRate.GetValue("User"))
vegf_pde.SetReferenceConcentration(1.e-9*mole_per_metre_cubed)

Add a boundary condition to fix the VEGF concentration in the vegf subdomain.



In [ ]:
vegf_boundary = microvessel_chaste.pde.DiscreteContinuumBoundaryCondition3()
vegf_boundary.SetType(microvessel_chaste.pde.BoundaryConditionType.IN_PART)
vegf_boundary.SetSource(microvessel_chaste.pde.BoundaryConditionSource.PRESCRIBED)
vegf_boundary.SetValue(3.e-9*mole_per_metre_cubed)
vegf_boundary.SetDomain(pellet)

Set up the PDE solvers for the vegf problem.



In [ ]:
vegf_solver = microvessel_chaste.pde.SimpleLinearEllipticFiniteElementSolver3()
vegf_solver.SetPde(vegf_pde)
vegf_solver.SetLabel("vegf")
vegf_solver.SetGrid(mesh)
vegf_solver.AddBoundaryCondition(vegf_boundary)

Set up an angiogenesis solver and add sprouting and migration rules.



In [ ]:
angiogenesis_solver = microvessel_chaste.simulation.AngiogenesisSolver3()
sprouting_rule = microvessel_chaste.simulation.OffLatticeSproutingRule3()
sprouting_rule.SetSproutingProbability(1.e-4* per_second)
migration_rule = microvessel_chaste.simulation.OffLatticeMigrationRule3()
migration_rule.SetChemotacticStrength(0.1)
migration_rule.SetAttractionStrength(0.5)
sprout_velocity = (50.0e-6/(24.0*3600.0))*metre_per_second #Secomb13
migration_rule.SetSproutingVelocity(sprout_velocity)
#angiogenesis_solver.SetMigrationRule(migration_rule)
#angiogenesis_solver.SetSproutingRule(sprouting_rule)
#sprouting_rule.SetDiscreteContinuumSolver(vegf_solver)
#migration_rule.SetDiscreteContinuumSolver(vegf_solver)
angiogenesis_solver.SetVesselNetwork(network)
angiogenesis_solver.SetBoundingDomain(cornea)

Set up the `MicrovesselSolver` which coordinates all solves.



In [ ]:
microvessel_solver = microvessel_chaste.simulation.MicrovesselSolver3()
microvessel_solver.SetVesselNetwork(network)
microvessel_solver.AddDiscreteContinuumSolver(vegf_solver)
microvessel_solver.SetOutputFileHandler(file_handler)
microvessel_solver.SetOutputFrequency(1)
microvessel_solver.SetAngiogenesisSolver(angiogenesis_solver)
microvessel_solver.SetUpdatePdeEachSolve(False)

Set up plotting



In [ ]:
scene.GetDiscreteContinuumMeshActorGenerator().SetVolumeOpacity(0.3)
scene.GetDiscreteContinuumMeshActorGenerator().SetDataLabel("Nodal Values")
scene.GetVesselNetworkActorGenerator().SetEdgeSize(5.0)
scene_modifier = microvessel_chaste.visualization.JupyterMicrovesselSceneModifier3(nb_manager)
scene_modifier.SetVtkScene(scene)
scene_modifier.SetUpdateFrequency(1)
microvessel_solver.AddMicrovesselModifier(scene_modifier)

Set the simulation time and run the solver.



In [ ]:
chaste.cell_based.SimulationTime.Instance().SetEndTimeAndNumberOfTimeSteps(60.0, 5)
microvessel_solver.Run()

Dump the parameters to file for inspection.



In [ ]:
ParameterCollection.Instance().DumpToFile(file_handler.GetOutputDirectoryFullPath()+"parameter_collection.xml")
nb_manager.add_parameter_table(file_handler)
# Tear down the test 
chaste.cell_based.TearDownNotebookTest()